This is a live notebook with experimental code to develop functionality for feature extraction.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'


['debug_windowing_data.ipynb', 'testing_mlpwrapper.ipynb', 'logs', 'gait_data_features.ipynb', 'feature_development.ipynb', 'data', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'featured_eeg_prediction.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'feature_correlation_analysis.ipynb', 'featured_eeg_prediction_gradient_boosting.ipynb', 'Analyse_results.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'featured_E4_prediction.ipynb', 'data_investigation_scratch.ipynb', 'grouping_and_crossvalidation.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH


from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from prepare_evaluation_data import load_dataframe_and_config


In [5]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

## Load data



In [6]:
subdir = 'E4_4secs'
windowed_data_dir = os.path.join(WINDOWED_BASE_PATH,subdir)

windowed_df, windowed_config = load_dataframe_and_config(
    windowed_data_dir, 'windowed.csv')

In [7]:
np.unique(windowed_df['participant'])

array(['VG_01', 'VG_03', 'VG_05', 'VG_06', 'VG_07', 'VG_08', 'VG_09',
       'VG_10', 'VH_01', 'VH_02', 'VH_03'], dtype=object)

In [11]:
s = set([1,2,3,4,4])
s.remove(4)
s

{1, 2, 3}

In [ ]:
n_channels = int(windowed_config['LOAD']['n_channels'])
channels = json.loads(windowed_config['LOAD']['channels'].replace("'",'"'))
participant_list = json.loads(windowed_config['LOAD']['participant_list'].replace("'",'"'))
Fs = int(windowed_config['LOAD']['sample_rate'])
window_size = int(windowed_config['LOAD']['window_size'])
time = window_size/Fs
print(f"Fs: {Fs}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")

In [ ]:
featured_data_dir = os.path.join(FEATURED_BASE_PATH,subdir)

featureddf, featuredconfig = load_dataframe_and_config(
    featured_data_dir, 'featured.csv')

# Aggregate functions

Initially targetted the following list taken from:
Public Domain Dataset for Human Activity Recognition Using Smartphones, (Anguita et al., 2013). 

Most now implemented. No energy bands 18/12/23. No angle between vectors 18/12/23. No SMA, 18/12/23/

| Function	|	Description |
|  :---:	|	:---:  |
| mean	|	Mean value |
| std	|	Standard deviation |
| mad	|	Median absolute value |
| max	|	Largest values in array |
| min	|	Smallest value in array |
| sma	|	Signal magnitude area |
| energy	|	Average sum of the squares |
| iqr	|	Interquartile range |
| entropy	|	Signal Entropy |
| arCoeff	|	Autorregresion coefficients |
| correlation	|	Correlation coefficient |
| maxFreqInd	|	Largest frequency component |
| meanFreq	|	Frequency signal weighted average |
| skewness	|	Frequency signal Skewness |
| kurtosis	|	Frequency signal Kurtosis |
| energyBand	|	Energy of a frequency interval |
| angle	|	Angle between two vectors |

Additional things added:

| Function	|	Description | Sources |
|  :---:	|	:---:  |	:---:  |
| Hurst exponent	|	Limit ratio of spread from random spread | [here](https://www.mdpi.com/1099-4300/23/12/1672) |
| Lyapunov exponent (spectrum)	|	Largest change | [here](https://www.mdpi.com/1099-4300/23/12/1672) |




# New Features

## Hurst Exponent

In [ ]:
from hurst import compute_Hc
series = [-0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0004891355764095, -0.0004629968719005, -0.0004373311970702, -0.0004139391165026, -0.0003926527809567, -0.0003737163347829, -0.0003583504997329, -0.0003474860761425, -0.0003402227817196, -0.0003345769436179, -0.000328793774319, -0.0003210727092393, -0.0003098268101014, -0.0002949034866865, -0.0002783321889066, -0.0002626764324406, -0.0002492790112153, -0.0002398031586175, -0.0002366445410849, -0.0002405203326466, -0.0002486533913176, -0.0002557335774776, -0.0002568017090104, -0.0002492790112153, -0.0002340810254062, -0.0002156786449988, -0.0002003585870145, -0.0001938124666208, -0.0001980544747081, -0.0002093614099336, -0.0002201342793926, -0.0002243457694361, -0.000221110856794, -0.0002123369192034, -0.0001995193408102, -0.0001833447775997, -0.0001657053482871, -0.0001506294346532, -0.0001415503166247, -0.0001383611810482, -0.0001374303807125, -0.0001331273365377, -0.0001207217517357, -9.92370489051652e-05, -7.261005569542993e-05, -4.695963988708325e-05, -2.487983520256353e-05, -4.661631189440771e-06, 1.537346456092163e-05, 3.46608682383459e-05, 5.272755016403447e-05, 7.125200274662392e-05, 9.26756694895857e-05, 0.0001168154421301, 0.0001399023422598, 0.0001587472343022, 0.0001741741054398, 0.000189356832227, 0.0002065079728389, 0.0002244678416113, 0.0002400015259021, 0.0002512626840619, 0.0002578850995651, 0.0002601739528496, 0.0002592736705577, 0.0002575341420614, 0.0002585717555504, 0.0002653620202944, 0.0002786984054322, 0.0002972228580148, 0.0003181734950789, 0.0003382085908293, 0.000355680170901, 0.0003726939803158, 0.0003917067215991, 0.0004112077515831, 0.0004268024719615, 0.0004356832227054, 0.0004399862668802, 0.0004453574425879, 0.0004554894331273, 0.0004695887693598, 0.0004852903028915, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004999999999999, 0.0004943541618982, 0.0004716029602502, 0.0004498283360036, 0.0004293354695963, 0.0004124589913786, 0.000401762417029, 0.0003956588082703, 0.0003874036774242, 0.0003694743266956, 0.0003390173189898, 0.0003000457770656, 0.0002613488975356, 0.0002308003356984, 0.0002097276264591, 0.0001944991226062, 0.0001811779964904, 0.0001677805752651, 0.000155237659266, 0.0001452887769893, 0.0001379797055008, 0.0001312352178225, 0.0001220798046845, 0.0001090180819409, 9.230945296406498e-05, 7.224383916990919e-05, 4.975204089417867e-05, 2.7489127946898587e-05, 7.133592736705562e-06, -1.3954375524528893e-05, -3.917753871976807e-05, -6.733043411917298e-05, -9.415579461356528e-05, -0.000116678110933, -0.0001361943999389, -0.0001558937972075, -0.00017650873579, -0.0001959487296864, -0.0002124132143129, -0.0002273823147936, -0.0002446097505149, -0.0002664911879148, -0.000291271839475, -0.0003130159456778, -0.0003261997405966, -0.0003303654535744, -0.0003303501945525, -0.000333524071107, -0.000345197222858, -0.0003644846265354, -0.0003845960173952, -0.0003993057145037, -0.0004080949111161, -0.000413878080415, -0.0004193255512321, -0.0004255664911879, -0.0004324483100633, -0.000439131761654, -0.0004448233768215, -0.0004505912870984, -0.0004597619592584, -0.0004750209811551, -0.0004958648050659, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0005, -0.0004739681086442, -0.0004489738307774, -0.0004298847943846, -0.0004189745937285, -0.0004160601205462, -0.0004171587701228, -0.0004179522392614, -0.000416838330663, -0.0004131609063859, -0.0004057907988097, -0.0003934615091172, -0.0003751049057755, -0.0003507972838941, -0.0003235751888303, -0.0002982909895475, -0.0002772030212863, -0.0002601586938277, -0.000247402151522, -0.0002385061417563, -0.0002304036011291, -0.0002190051117723, -0.0002031815060654, -0.0001860761425192, -0.0001725261310749, -0.0001652628366521, -0.0001634622720683, -0.0001632639047837, -0.0001591897459372, -0.0001462348363469, -0.0001227664606698, -9.296559090562296e-05, -6.563668268863966e-05, -4.781414511329825e-05, -3.90707255664912e-05, -3.2906080720225846e-05, -2.304875257495996e-05, -7.896543831540412e-06, 1.140611886778056e-05, 3.401998931868465e-05, 5.902952620737009e-05, 8.374914168001829e-05, 0.0001032501716639, 0.0001124513618677, 0.0001097810330357, 9.874876020447086e-05, 8.617532616159302e-05, 7.926298924238954e-05, 8.307774471656365e-05, 9.702449073014418e-05, 0.0001139772640573, 0.0001254978255893, 0.0001279697871366, 0.0001249790188448, 0.0001240329594872, 0.0001297856107423, 0.000140177004654, 0.0001487830930037, 0.0001512245365072, 0.000149179827573, 0.0001497444113832, 0.0001599069199664, 0.0001803082322423, 0.0002045090409704, 0.0002241474021515, 0.0002343099107347, 0.0002352407110704, 0.0002309376668955, 0.0002248950942244, 0.0002168688487067, 0.0002050888838025, 0.0001911726558327, 0.0001790264744029, 0.0001690012970168, 0.0001569924467841, 0.000140619516289, 0.0001230869001297, 0.0001100709544518, 0.0001049897001602, 0.0001062714579995, 0.000108117799649, 0.0001056763561455, 9.980163271534292e-05, 9.581902800030516e-05, 9.653620202944988e-05, 9.931334401464864e-05, 9.99084458686198e-05, 9.574273289082169e-05, 8.7533379110399e-05, 8.040741588464178e-05, 8.098725871671624e-05, 9.121080338750283e-05, 0.0001070038910505, 0.0001235751888303, 0.00014046692607, 0.0001616464484626, 0.0001903028915846, 0.000224956130312, 0.0002606927595941, 0.0002917753871976, 0.0003130006866559, 0.0003229343099107, 0.000325543602655, 0.0003270695048447, 0.0003319066147859, 0.0003417181658655, 0.0003558785381857, 0.0003721446555275, 0.0003885175860227, 0.0004038834210727, 0.0004169451438162, 0.0004278553444724, 0.0004399710078583, 0.0004563286793316, 0.0004746089875638, 0.0004881132219424, 0.000491134508278, 0.0004831235217822, 0.0004693598840314, 0.0004573815518425, 0.0004515831235217, 0.0004520714122224, 0.0004560082398718, 0.0004574883649958, 0.0004498741130693, 0.0004311207751583, 0.0004055466544594, 0.0003803845273517, 0.0003605935759517, 0.0003460212100404, 0.0003317998016327, 0.0003115815976195, 0.0002833524071107, 0.0002520408941786, 0.0002251697566186, 0.0002065842679484, 0.0001942549782558, 0.0001835736629282, 0.0001719005111772, 0.0001597848477912, 0.0001486305027847, 0.0001377965972381, 0.0001242008087281, 0.0001050507362478, 8.069733730067901e-05, 5.432974746318759e-05, 2.813000686655984e-05, 1.1825741969939583e-06, -2.7656977187762275e-05, -5.663385976958878e-05, -8.263523308155948e-05, -0.0001045014114595, -0.0001234378576333, -0.0001424658579385, -0.0001645761806668, -0.0001899671931029]
series = np.array(series)
compute_Hc(series, kind="random_walk", min_window=10, max_window=None, simplified=False)

In [ ]:
from hurst import __to_inc
start = 0
w = 562
tmp_series = series[start:start+w]
incs = __to_inc(tmp_series)
mean_inc = (tmp_series[-1] - tmp_series[0]) / len(incs)
deviations = incs - mean_inc
Z = np.cumsum(deviations)
R = max(Z) - min(Z)
S = np.std(incs, ddof=1)
print(f"Z: {Z}\nR: {R}, S={S}")
R/S

In [ ]:
from hurst import __get_simplified_RS, __get_RS
import warnings
import math
kind="random_walk"
min_window=10
max_window=None
simplified=True
max_window = None
max_window = max_window or len(series)-1
window_sizes = list(map(
    lambda x: int(10**x),
    np.arange(math.log10(min_window), math.log10(max_window), 0.25)))
window_sizes.append(len(series))

if simplified:
    RS_func = __get_simplified_RS
else:
    RS_func = __get_RS

RS = []
for w in window_sizes:
    rs = []
    for start in range(0, len(series), w):
        if (start+w)>len(series):
            break
        _ = RS_func(series[start:start+w], kind)
        if _ != 0:
            rs.append(_)
    print(f"w = {w}, rs = {rs}")
    try:
        RS.append(np.mean(rs))
    except FloatingPointError:
        warnings.warn("FloatingPointError encountered in compute_Hc.")
RS

In [ ]:
__get_simplified_RS(series[0:562], kind)

In [ ]:
__get_RS(series[0:562], kind)

In [ ]:
w = 562
for start in range(0, len(series), w):
    print(f"start = {start}")
    print(f"(start+w)>len(series) = {(start+w)>len(series)}")
    if (start+w)>len(series):
        break
    _ = RS_func(series[start:start+w], kind)
    if _ != 0:
        rs.append(_)


In [ ]:
windowed_df.iloc[4387,:]

## Things to explore

THe following list is a live list of things that might potentially relevant.

| Function	|	Description | Sources |
|  :---:	|	:---:  |	:---:  |
| Heart rate variability | A number of choices | |
| Lempev-Ziv Complexity	| LZ complexity of encoding of binary sequence. | TBC |
| Durbin-Watson test statistic | | [here](https://en.wikipedia.org/wiki/Durbin%E2%80%93Watson_statistic) |


In [ ]:
from predicament.data.features import convert_timeseries_to_features
feature_set = set(
    ['Mean', 'SD', 'MAD', 'Max', 'Min',# 'SMA',
     'Energy', 'IQR', 'SampleEntropy',
    'arCoeff', 'Correlation', 'MaxFreqInd', 'MeanFreq', 'FreqSkewness',
    'FreqKurtosis'#, 'EnergyBands'
    ])
features111, feature_names = convert_timeseries_to_features(
        X111, feature_set,
        entropy_tol=0.05*(np.mean(np.std(X111,axis=1))))
print(f"features111.shape = {features111.shape}")
print(f"feature_names.shape = {len(feature_names)}")
print(f"feature_names = {feature_names}")


### Lempel-Ziv entropy

In [ ]:
# # adapted from:
# # https://stackoverflow.com/questions/46296891/entropy-estimator-based-on-the-lempel-ziv-algorithm-using-python
# import math
# def contains(small, big):
#     try:
#         big.tobytes().index(small.tobytes())//big.itemsize
#         return True
#     except ValueError:
#         return False

# def lempel_ziv_entropy(l):
#     """
#     input
#     -----
#     l - a 1d numpy timeseries array 
#     """
#     n = len(l)
#     sum_gamma = 0

#     for i in range(1, n):
#         sequence = l[:i]

#         for j in range(i+1, n+1):
#             s = l[i:j]
#             if contains(s, sequence) != True:
#                 sum_gamma += len(s)
#                 break

#     ae = 1 / (sum_gamma / n) * math.log(n)
#     return ae




In [ ]:
for c in range(X111.shape[0]):
    ent_raw = lempel_ziv_entropy(X111[c,:])
    print(f"ent_raw = {ent_raw}")
    ent_str = lempel_ziv_entropy(X111[c,:].astype(str))
    print(f"ent_str = {ent_str}")
    

In [ ]:
# def quantize_timeseries(seq, amp_min, amp_max, bits_per=8, binarize=True):
#     bins = np.linspace(amp_min, amp_max, 2**bits_per+1)
# #     print(f"bins = {bins}")
#     if bits_per <= 8:
#         quantized = np.empty(seq.size, dtype=np.uint8)
# #     elif bits_per == 32:
# #         quantized = np.empty(seq.size, dtype=np.int32)
#     else:
#         raise ValueError(f'Invalid bits per {bits_per}')
#     for i in range(2**bits_per):
# #         print(f"i = {i}")
# #         print(f"sum((seq >= bins[i])) = {sum((seq >= bins[i]))}")
# #         print(f"sum((seq < bins[i+1])) = {sum((seq < bins[i+1]))}")
#         quantized[(seq >= bins[i]) & (seq < bins[i+1]) ] = i
#     if binarize:
#         unpacked = np.unpackbits(quantized)
#         if bits_per < 8:
#             # by default there are 8 bits per unpacked uint8. We can make this shorter
#             # by removing the big end of each encoding.
#             unpacked =  unpacked.reshape((-1,8))[:,-bits_per:].flatten()
#         return unpacked
#     else:
#         return quantized


# def quantize_multichannel(X, amp_mins, amp_maxes, bits_per=8, binarize=True):
#     bins = np.linspace(amp_mins, amp_maxes, 2**bits_per+1, axis=1)
# #     print(f"bins = {bins}")
#     if bits_per <= 8:
#         quantized = np.empty(X.shape, dtype=np.uint8)
#     else:
#         raise ValueError(f'Invalid bits per {bits_per}')
#     for i in range(2**bits_per):
# #         print(f"i = {i}")
# #         print(f"sum((seq >= bins[i])) = {sum((seq >= bins[i]))}")
# #         print(f"sum((seq < bins[i+1])) = {sum((seq < bins[i+1]))}")
#         quantized[(X >= bins[:,i:i+1]) & (X < bins[:,i+1:i+2]) ] = i
# #     print(f"quantized = {quantized}")
#     if binarize:
#         n_channels = quantized.shape[0]
#         unpacked = np.unpackbits(quantized, axis=1)
#         if bits_per < 8:
#             # by default there are 8 bits per packed quantized vector. We can make this shorter
#             print(f"unpacked.reshape((n_channels, -1,8)).shape = {unpacked.reshape((n_channels, -1,8)).shape}")
#             unpacked =  unpacked.reshape((n_channels, -1,8))[:,:,-bits_per:].reshape((n_channels, -1))
#         return unpacked
#     else:
#         return quantized





In [ ]:
data = windowed_df.iloc[:,3:].to_numpy()
N = data.shape[0]
data_tens = data.reshape((N, n_channels, -1))
amp_mins = np.min(data_tens, axis=(0,2))
amp_maxes = np.max(data_tens, axis=(0,2))
i = 0
X = data[i].reshape((n_channels,-1))
X.shape

In [ ]:
c = 0
X[c,:10]

In [ ]:
bits_per = 4
quantized = quantize_timeseries(X[c,:10], amp_mins[c], amp_maxes[c], bits_per=bits_per, binarize=False)
quantized

In [ ]:

unpacked = np.unpackbits(quantized, axis=0)
unpacked

In [ ]:
unpacked.reshape((-1,8))[:,-bits_per:].flatten()

In [ ]:
np.unpackbits(np.array([7], dtype=np.uint8))

In [ ]:
print(f"amp_mins = {amp_mins}")
print(f"amp_maxes = {amp_maxes}")

bins = np.linspace(amp_mins, amp_maxes, 3, axis=1)
bins

In [ ]:
Q = np.vstack([quantize_timeseries(X[c,:10], amp_mins[c], amp_maxes[c], bits_per=4, binarize=True) for c in range(n_channels)])
Q.shape

In [ ]:
Q[:,:]

In [ ]:
Qalt = quantize_multichannel(X[:,:10], amp_mins, amp_maxes, bits_per=4, binarize=True)
Qalt.shape

In [ ]:
np.all(Q == Qalt)

In [ ]:
help(np.unpackbits)

In [ ]:
for c in range(X111.shape[0]):
    x = X111[c,:]
    amp_max = x.max()
    amp_min = x.min()
#     print(f"x = {x}")
    quantized =  quantize_timeseries(x, amp_min, amp_max, bits_per=1)
    lze = lempel_ziv_entropy(quantized)
    print(f"lze = {lze}")

In [ ]:
for i in range(3):
    seq = X111[0,0:i]
    print(f"i={i}, seq = {seq}")
    print(f"seq.tobytes() = {seq.tobytes()}")
    print(f"seq.astype(str) = {seq.astype(str)}")
    print(f"seq.astype(str).tobytes() = {seq.astype(str).tobytes()}")
seq = np.array([4.68288097])
print(f"seq = {seq}")
print(f"seq.tobytes() = {seq.tobytes()}")
seq = np.array([4], dtype=int)
print(f"seq = {seq}")
print(f"seq.tobytes() = {seq.tobytes()}")
seq = np.array([5], dtype=int)
print(f"seq = {seq}")
print(f"seq.tobytes() = {seq.tobytes()}")


In [ ]:
def int_to_binary():
    pass

In [ ]:
import numpy as np
np.binary_repr(5, width=8)

In [ ]:
a = np.array([3,6,9], dtype=np.uint8)
np.unpackbits(a)

In [ ]:

np.unpackbits(a).tobytes()

In [ ]:
a = np.array([[3,6,9],[6,9,3],[9,3,6]], dtype=np.uint8)
np.unpackbits(a).shape

In [ ]:
windowed_data = windowed_df.iloc[:,3:].to_numpy()
windowed_data.shape

In [ ]:
# import numpy as np
# from itertools import combinations
# from math import log

# def sample_entropy_stack(timeseries_data, window_size, rs):
#     templates = construct_templates_stack(timeseries_data, window_size)
#     B = get_matches_stack(templates, rs)
    
#     templates_m1 = construct_templates_stack(timeseries_data, window_size + 1)
#     A = get_matches_stack(templates_m1, rs)
    
#     return -np.log(A / B)

# def construct_templates_stack(timeseries_data, m=2):
#     window_shape = (timeseries_data.shape[0], m)
#     return np.squeeze(np.lib.stride_tricks.sliding_window_view(timeseries_data, window_shape))

# def get_matches_stack(templates, rs):
#     num_rows, _, _ = templates.shape
#     indices = np.arange(num_rows)
#     combs = np.array(list(combinations(indices, 2)))
#     matches = np.sum(np.all(np.abs(templates[combs[:, 0]] - templates[combs[:, 1]]) < rs[np.newaxis, :, np.newaxis], axis=2), axis=0)
#     return matches

# # Example usage:
# shape = (3,20)
# timeseries_data_2d = np.sin(np.arange(shape[0])[:,np.newaxis]*np.pi*np.linspace(0,1,shape[1])) + 0.01*np.random.random(shape)

# window_size = 2
# rs = 0.2*np.ones(3)

# result = sample_entropy_stack(timeseries_data_2d, window_size, rs)
# print(result)

In [ ]:
feature_set1 = ["Mean", "Correlation", "MaxFreqInd", "arCoeff", "FreqKurtosis", "SD", "Hurst", "IQR", "FreqSkewness", "LempelZivComplexity", "LyapunovExponent", "MeanFreq", "MAD", "Min", "Max"]
feature_set2 = ["Mean", "LempelZivComplexity", "Max", "IQR", "MeanFreq", "Correlation", "Min", "arCoeff", "MaxFreqInd", "LyapunovExponent", "MAD", "FreqKurtosis", "FreqSkewness", "Hurst", "Energy", "SD"]

feature_set1 = set(feature_set1)
feature_set2 = set(feature_set2)

feature_set2 - feature_set1